# Bert를 사용한 문장 간 관계수치 예측

2개의 문장 간의 관계를 수치로 에측한다.

0 : 무관
5 : 상관

# 필요 라이브러리 설치

In [1]:
!pip install transformers==3.0.2
!pip install sentencepiece

     |████████████████████████████████| 769 kB 7.4 MB/s 
     |████████████████████████████████| 3.0 MB 44.4 MB/s 
     |████████████████████████████████| 1.2 MB 64.7 MB/s 
     |████████████████████████████████| 895 kB 75.1 MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [3]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'

# 데이터

## 데이터 다운로드

In [4]:
!git clone https://github.com/kakaobrain/KorNLUDatasets

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [10]:
!wc ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv

  550153  8595590 78486224 ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv


## 데이터 로딩

In [122]:
df = pd.read_csv("KorNLUDatasets/KorSTS/sts-train.tsv", delimiter = '\t', quoting = 3)

In [143]:
df.head(10)

,genre,filename,year,id,score,sentence1,sentence2
0,main-news,headlines,2013,237,3.8,프랑스 사회주의 정당이 의회 승리,프랑스 사회주의자들이 절대 의회 다수를 차지하다
1,main-news,headlines,2013,477,4.2,남아프리카 경찰은 30명의 광부들을 총으로 쏘았다.,남아프리카 경찰은 그들이 30명 이상의 광부들을 죽였다고 말한다.
2,main-captions,MSRvid,2012test,582,2.5,남자가 닭고기를 용기에 넣고 있다.,한 남자가 상자에 음식을 넣고 있다.
3,main-captions,images,2014,21,3.0,노란 꽃 앞에 서 있는 검은 개,들판에 서 있는 검은 개
4,main-forum,deft-forum,2014,373,4.6,세례자 존은 출생시부터 나자라이트였다.,세례 요한은 출생 때부터 나자라이트였다.
5,main-captions,MSRvid,2012train,665,0.0,동물이 걷고 있다.,한 여성이 눈 화장을 하고 있다.
6,main-news,headlines,2015,591,4.0,베네수엘라 감옥에서 폭력으로 16명 사망,베네수엘라 감옥에서 16명 사망
7,main-captions,MSRvid,2012test,52,2.4,남자가 빵을 자르고 있다.,남자가 양파를 자르고 있다.
8,main-captions,MSRvid,2012train,143,5.0,계단을 내려가는 한 남자.,한 남자가 계단을 걸어 내려간다.
9,main-news,headlines,2014,629,3.8,베네수엘라의 후고 차베스는 암으로 사망한다.,베네수엘라 대통령 휴고 차베스가 58세에 암으로 사망하다.


## 데이터 섞기

In [124]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,genre,filename,year,id,score,sentence1,sentence2
0,main-news,headlines,2013,237,3.8,프랑스 사회주의 정당이 의회 승리,프랑스 사회주의자들이 절대 의회 다수를 차지하다
1,main-news,headlines,2013,477,4.2,남아프리카 경찰은 30명의 광부들을 총으로 쏘았다.,남아프리카 경찰은 그들이 30명 이상의 광부들을 죽였다고 말한다.
2,main-captions,MSRvid,2012test,582,2.5,남자가 닭고기를 용기에 넣고 있다.,한 남자가 상자에 음식을 넣고 있다.
3,main-captions,images,2014,21,3.0,노란 꽃 앞에 서 있는 검은 개,들판에 서 있는 검은 개
4,main-forum,deft-forum,2014,373,4.6,세례자 존은 출생시부터 나자라이트였다.,세례 요한은 출생 때부터 나자라이트였다.


## 필요 입출력 값 준비

In [126]:
sentences1 = df.sentence1.values.copy().astype(np.str)
sentences2 = df.sentence2.values.copy().astype(np.str)
labels = df.score.values.copy().astype(np.float)/5.0

In [127]:
print(sentences1.shape)
print(sentences2.shape)
print(labels.shape)

(5749,)
(5749,)
(5749,)


필요 시, 실습 시간 관계로 전체 중에 일부 만 사용한다.

In [128]:
COUNT = 10000
sentences1 = sentences1[:COUNT]
sentences2 = sentences2[:COUNT]
labels = labels[:COUNT]

## 토큰나이저 생성

In [129]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=False, model_max_length=SEQ_LENGTH)

In [130]:
encoded_tokens= tokenizer.encode("하늘이 푸르다.", text_pair="파란색이 좋아.")
print(encoded_tokens)
print(tokenizer.convert_ids_to_tokens(encoded_tokens))

[101, 9952, 118762, 10739, 9935, 31401, 11903, 119, 102, 9901, 49919, 41442, 10739, 9685, 16985, 119, 102]
['[CLS]', '하', '##늘', '##이', '푸', '##르', '##다', '.', '[SEP]', '파', '##란', '##색', '##이', '좋', '##아', '.', '[SEP]']


In [131]:
tokenized = tokenizer("하늘이 푸르다.", text_pair="파란색이 좋아.", max_length=20, padding='max_length')
print(tokenizer.decode(tokenized['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
print(tokenized['input_ids'])
print(tokenized['attention_mask'])
print(tokenized['token_type_ids'])

[CLS] 하늘이 푸르다. [SEP] 파란색이 좋아. [SEP] [PAD] [PAD] [PAD]
['[CLS]', '하', '##늘', '##이', '푸', '##르', '##다', '.', '[SEP]', '파', '##란', '##색', '##이', '좋', '##아', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
[101, 9952, 118762, 10739, 9935, 31401, 11903, 119, 102, 9901, 49919, 41442, 10739, 9685, 16985, 119, 102, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]


## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [132]:
import logging
logging.basicConfig(level=logging.ERROR)

In [133]:
def build_model_input(sentences1, sentences2):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for sentence1, sentence2 in zip(sentences1, sentences2):
    tokenized = tokenizer(sentence1, text_pair=sentence2, max_length=SEQ_LENGTH, padding='max_length')
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids))


In [134]:
x = build_model_input(sentences1, sentences2)
y = labels

In [135]:
print(x[0].shape)

(5749, 128)


## train/test 분리

In [136]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

# 학습

## 모델 생성

In [138]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertPredictor(tf.keras.Model):
  def __init__(self):
    super(TFBertPredictor, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.predcitor = Dense(1, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range))

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.predcitor(v)

    return out

model = TFBertPredictor()


참고로 Bert의 default 설정은 다음과 같다.

In [114]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [139]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])


## 학습 실행

In [140]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
130/130 [==============================] - 123s 518ms/step - loss: 0.0557 - mae: 0.1864 - val_loss: 0.0368 - val_mae: 0.1500
Epoch 2/5
130/130 [==============================] - 65s 497ms/step - loss: 0.0354 - mae: 0.1485 - val_loss: 0.0288 - val_mae: 0.1312
Epoch 3/5
130/130 [==============================] - 65s 497ms/step - loss: 0.0258 - mae: 0.1271 - val_loss: 0.0356 - val_mae: 0.1491
Epoch 4/5
130/130 [==============================] - 65s 497ms/step - loss: 0.0194 - mae: 0.1093 - val_loss: 0.0316 - val_mae: 0.1356
Epoch 5/5
130/130 [==============================] - 65s 498ms/step - loss: 0.0156 - mae: 0.0988 - val_loss: 0.0308 - val_mae: 0.1330


In [141]:
loss, mape = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("mape =", mape)

36/36 [==============================] - 6s 163ms/step - loss: 0.0304 - mae: 0.1318
loss = 0.030417706817388535
mape = 0.13178132474422455


## 예측 실행

In [145]:
def do_classify(sentence1, sentence2):
  model_input = build_model_input([sentence1], [sentence2])
  y_ = model.predict(model_input)
  print(sentence1, sentence2, "-->", "score :",y_[0])

do_classify("나는 왜 그런지 잘 모르겠다.", "하늘이 푸르다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런 일이 일어났는지 모르겠어.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런지 완전히 모르겠어.")

나는 왜 그런지 잘 모르겠다. 하늘이 푸르다. --> score : [0.10910998]
나는 왜 그런지 잘 모르겠다. 나는 왜 그런 일이 일어났는지 모르겠어. --> score : [0.5142913]
나는 왜 그런지 잘 모르겠다. 나는 왜 그런지 완전히 모르겠어. --> score : [0.6512094]
